Este cuaderno es simplemente una versión anotada en Jupyter del programa CSDALindenberg.jl. 

Usa el operador convexo de Lindenberg como Laplaciano.
Para usarse requiere un archivo *jld que contenga el LFP
como fue registrado en el experimento (o promediado sobre eventos evocados),
y, optativamente, una lista de electrodos inusables ("saturados").

In [1]:
using JLD
include("LindenbergOperadores.jl")
importall LindenbergOperadores

In [3]:
arxnombre="../JLDFiles/estriadoscachos/Corte_final_estriado_slice1_1004AP.jld"
arx=load(arxnombre)

Dict{String,Any} with 6 entries:
  "Canalesrespuesta" => Set(Array{Int64,1}[[30, 55], [52, 14], [6, 37], [38, 4]…
  "desde"            => 29000
  "hasta"            => 34000
  "LFPSaturados"     => [4125.0 -12.085 … 6.04248 22.1558; -34.2407 -8.05664 … …
  "freq"             => 17.8555
  "CanalesSaturados" => Set(Array{Int64,1}[[64, 63], [9, 38], [9, 41], [9, 37],…

In [4]:
#quitemos la terminacion de los nombres de archivo
palabra=replace(arxnombre,".jld", "")
println(palabra)

../JLDFiles/estriadoscachos/Corte_final_estriado_slice1_1004AP


In [5]:
# cargando los datos importantes
if haskey(arx, "LFPSaturados")
    LFP=arx["LFPSaturados"]
else
    LFP=arx["LFPTotal"]
end
saturados=arx["CanalesSaturados"]
respuestas=arx["Canalesrespuesta"];
frecuencia=arx["freq"]

17.85550205219098

In [6]:
lfpParchado=copy(LFP)
#Poner en cero los canales inservibles
for m in saturados
    q=m[1]
    p=m[2]
    lfpParchado[q,p,:]=0
end

In [7]:
listaredux=TiraOrillas(saturados)


Set(Any[[22, 63], [9, 38], [9, 41], [9, 37], [10, 41], [8, 37], [9, 31], [11, 41], [9, 36], [7, 30]  …  [55, 57], [8, 40], [10, 40], [8, 41], [13, 11], [23, 63], [9, 39], [10, 38], [10, 39], [8, 31]])

In [8]:
#Creamos una mancha suave sobre el canal saturado.
for m in listaredux
        q=m[1]
        p=m[2]
        vecinos=vecindad8(m)
        lfpParchado[q,p,:]=promediasobreconjunto(vecinos,lfpParchado)
end


In [9]:
size(lfpParchado)

(64, 64, 5001)

In [10]:
#en realidad nunca haces referencia al tercer numero como tiempo
(mu,nu,lu)=size(lfpParchado)
#Aplicamos un suavizado Gaussiano Temporal (esto afecta mucho las animaciones)
lfpplanchado=zeros(mu,nu,lu)
for j=1:mu,l=1:nu
    porromponpon=vec(lfpParchado[j,l,:])
    lfpplanchado[j,l,:]=GaussSuavizarTemporal(porromponpon)
end



In [11]:

aux1=zeros(mu,nu,lu)
aux2=zeros(mu,nu,lu)
#Suavizamos espacialmente con un filtro Gaussiano bidimensional el LFP.
#Posteriormente sacamos el dCSD.
for t=1:lu
    aux1[:,:,t]=GaussianSmooth(lfpplanchado[:,:,t])
    aux2[:,:,t]=DiscreteLaplacian(aux1[:,:,t])
end
CSD=-aux2;


In [12]:

#Descartamos variables auxiliares
lfpParchado=0
aux1=0
aux2=0

#Observa que vas a guardar en el mismo archivo todo. 
paguardar=load(arxnombre)
paguardar["CSDLindenberg"]=CSD
save(arxnombre,paguardar)

